# **Portofolio 2**

In [178]:
from dotenv import load_dotenv
import os
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

#Bigquery Connection
from google.oauth2 import service_account
import pandas_gbq

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [179]:
BASE_URL = "https://www.last.fm/api"

In [180]:
# Configuration 
LASTFM_API_KEY = os.getenv('LASTFM_API_KEY')

In [181]:
# Last.FM API base URL
LASTFM_API_URL = 'http://ws.audioscrobbler.com/2.0/'

In [182]:
# load the .env file
load_dotenv("D:\\Purwadhika\\Portofolio\\.env")

# read the key
LASTFM_API_KEY = os.getenv("LASTFM_API_KEY")
print(LASTFM_API_KEY)


5e037de975e0eb13e723be687f70ea98


## **Fungsi**

In [183]:
def get_top_tracks(limit=100, period='overall'):
    """
    Fetch top tracks globally from Last.fm
    
    Args:
        limit (int): Number of tracks to fetch
        period (str): Time period - overall, 12month, 6month, 3month, 1month, 7day
    
    Returns:
        list: List of top tracks
    """
    params = {
        'method': 'chart.gettoptracks',
        'api_key': LASTFM_API_KEY,
        'format': 'json',
        'limit': limit
    }
    
    try:
        print(f"   Fetching top {limit} global tracks...")
        response = requests.get(LASTFM_API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        
        tracks = data.get('tracks', {}).get('track', [])
        if isinstance(tracks, dict):
            tracks = [tracks]
        
        return tracks
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching top tracks: {e}")
        return []

Fungsi ini adalah utility yang robust untuk mengambil data lagu-lagu populer dari Last.fm dengan error handling yang baik, meskipun parameter period tidak digunakan dalam implementasi saat ini.

In [184]:
def get_top_artists(limit=100):
    """
    Fetch top artists globally from Last.fm
    
    Args:
        limit (int): Number of artists to fetch
    
    Returns:
        list: List of top artists
    """
    params = {
        'method': 'chart.gettopartists',
        'api_key': LASTFM_API_KEY,
        'format': 'json',
        'limit': limit
    }
    
    try:
        print(f"   Fetching top {limit} global artists...")
        response = requests.get(LASTFM_API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        
        artists = data.get('artists', {}).get('artist', [])
        if isinstance(artists, dict):
            artists = [artists]
        
        return artists
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching top artists: {e}")
        return []

Fungsi ini adalah utility yang simpel dan efektif untuk mengambil data artis populer dari Last.fm. Strukturnya hampir identik dengan get_top_tracks(), menunjukkan konsistensi dalam design pattern untuk API calls dalam aplikasi ini.

In [185]:
def get_top_tags(limit=100):
    """
    Fetch top tags (genres) globally from Last.fm
    
    Args:
        limit (int): Number of tags to fetch
    
    Returns:
        list: List of top tags
    """
    params = {
        'method': 'chart.gettoptags',
        'api_key': LASTFM_API_KEY,
        'format': 'json',
        'limit': limit
    }
    
    try:
        print(f"   Fetching top {limit} global genres...")
        response = requests.get(LASTFM_API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        
        tags = data.get('tags', {}).get('tag', [])
        if isinstance(tags, dict):
            tags = [tags]
        
        return tags
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching top tags: {e}")
        return []

Fungsi get_top_tags() adalah komponen penting untuk:

Genre Analysis: Memahami tren genre musik
Music Categorization: Mengkategorikan konten musik
Trend Discovery: Menemukan tren baru dalam musik
Recommendation Systems: Basis untuk sistem rekomendasi
Strukturnya konsisten dengan fungsi API lain dalam aplikasi, memudahkan maintenance dan menunjukkan good coding practices dengan pattern yang seragam.

In [186]:
def get_artist_info(artist_name):
    """
    Fetch detailed info about an artist including country
    
    Args:
        artist_name (str): Name of the artist
    
    Returns:
        dict: Artist information
    """
    params = {
        'method': 'artist.getinfo',
        'artist': artist_name,
        'api_key': LASTFM_API_KEY,
        'format': 'json'
    }
    
    try:
        response = requests.get(LASTFM_API_URL, params=params)
        response.raise_for_status()
        return response.json().get('artist', {})
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching artist info for {artist_name}: {e}")
        return {}

get_artist_info() adalah fungsi utility yang clean dan simple untuk mendapatkan informasi detail artis. Meskipun docstring menyebut "including country", fungsi ini tidak benar-benar mengembalikan data country karena keterbatasan Last.fm API. Fungsi ini cocok untuk:

Building artist profiles
Getting artist statistics
Finding similar artists
Extracting artist biography
Getting artist tags/genres
Strukturnya lebih sederhana dibanding fungsi get_top_*() karena hanya handle single entity, tapi tetap maintain error handling pattern yang konsisten.

In [187]:
def get_track_tags(artist_name, track_name):
    """
    Fetch genres/tags for a specific track
    
    Args:
        artist_name (str): Name of the artist
        track_name (str): Name of the track
    
    Returns:
        str: Comma-separated genre names
    """
    params = {
        'method': 'track.gettoptags',
        'artist': artist_name,
        'track': track_name,
        'api_key': LASTFM_API_KEY,
        'format': 'json'
    }
    
    try:
        response = requests.get(LASTFM_API_URL, params=params)
        response.raise_for_status()
        data = response.json()
        
        tags = data.get('toptags', {}).get('tag', [])
        if isinstance(tags, dict):
            tags = [tags]
        
        # Extract just the tag names (top 5 genres)
        tag_names = [tag.get('name') for tag in tags[:5]]
        return ', '.join(tag_names) if tag_names else 'Unknown'
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching tags for {track_name}: {e}")
        return 'Unknown'


get_track_tags() adalah fungsi utility yang specialized untuk mendapatkan genre/tags sebuah lagu dalam format yang langsung readable (string).

In [188]:
def organize_top_tracks(tracks):
    """
    organize top tracks into DataFrame with genres
    
    Args:
        tracks (list): List of track dictionaries
    
    Returns:
        pd.DataFrame: organized tracks data
    """
    organized_tracks = []
    
    for idx, track in enumerate(tracks, 1):
        try:
            artist_name = track.get('artist', {}).get('name') if isinstance(track.get('artist'), dict) else track.get('artist')
            track_name = track.get('name')
            
            # Fetch genre tags
            print(f"      Fetching genres for {track_name}...", end=' ', flush=True)
            genres = get_track_tags(artist_name, track_name)
            print("✓")
            time.sleep(0.1)  # Rate limiting
            
            organized_track = {
                'rank': idx,
                'track_name': track_name,
                'artist': artist_name,
                'genres': genres,
                'playcount': int(track.get('playcount', 0)),
                'listeners': int(track.get('listeners', 0)),
                'url': track.get('url'),
                'data_type': 'top_track',
                'extracted_at': datetime.utcnow()
            }
            organized_tracks.append(organized_track)
        
        except (KeyError, ValueError, TypeError) as e:
            print(f"Error parsing track: {e}")
            continue
    
    return pd.DataFrame(organized_tracks)

organize_top_tracks() adalah fungsi data transformation yang crucial dalam pipeline, bertindak sebagai bridge antara raw API data dan analytical-ready DataFrame.

In [189]:
def organize_top_artists(artists):
    """
    organize top artists into DataFrame with country info
    
    Args:
        artists (list): List of artist dictionaries
    
    Returns:
        pd.DataFrame: organized artists data
    """
    organized_artists = []
    
    for idx, artist in enumerate(artists, 1):
        try:
            artist_name = artist.get('name')
            
            # Fetch detailed info to get country
            print(f"      Fetching details for {artist_name}...", end=' ', flush=True)
            artist_info = get_artist_info(artist_name)
            
            country = None
            if artist_info and 'country' in artist_info:
                country = artist_info.get('country')
            
            print("✓")
            time.sleep(0.1)  # Rate limiting
            
            organized_artist = {
                'rank': idx,
                'artist_name': artist_name,
                'country': country,
                'playcount': int(artist.get('playcount', 0)),
                'listeners': int(artist.get('listeners', 0)),
                'url': artist.get('url'),
                'data_type': 'top_artist',
                'extracted_at': datetime.utcnow()
            }
            organized_artists.append(organized_artist)
        
        except (KeyError, ValueError, TypeError) as e:
            print(f"Error parsing artist: {e}")
            continue
    
    return pd.DataFrame(organized_artists)

organize_top_artists() memiliki fundamental flaw - mencoba mengambil data country yang tidak tersedia di Last.fm API

In [190]:
def organize_top_tags(tags):
    """
    organize top tags/genres into DataFrame
    
    Args:
        tags (list): List of tag dictionaries
    
    Returns:
        pd.DataFrame: organized tags data
    """
    organized_tags = []
    
    for idx, tag in enumerate(tags, 1):
        try:
            # Handle reach count
            reach_value = tag.get('reach', 0)
            
            organized_tag = {
                'rank': idx,
                'genre': tag.get('name'),
                'reach': int(reach_value) if reach_value else 0,
                'url': tag.get('url'),
                'data_type': 'top_genre',
                'extracted_at': datetime.utcnow()
            }
            organized_tags.append(organized_tag)
        
        except (KeyError, ValueError, TypeError) as e:
            print(f"Error parsing tag: {e}")
            continue
    
    return pd.DataFrame(organized_tags)


In [191]:
def main():
    """Main pipeline execution"""
    
    print("=" * 60)
    print("Last.fm Global Data → DataFrames")
    print("=" * 60)
    
    # Validate configuration
    if not LASTFM_API_KEY:
        print("Error: LASTFM_API_KEY not found in .env file.")
        return
    
    # Fetch and organize Top Tracks
    print("\n🎵 FETCHING TOP TRACKS")
    print("-" * 60)
    top_tracks = get_top_tracks(limit=100)
    if top_tracks:
        df_tracks = organize_top_tracks(top_tracks)
        
        print(f"   organized {len(df_tracks)} tracks")
        print(f"\n   Sample:")
        print(df_tracks[['rank', 'track_name', 'artist', 'genres', 'playcount']].head(10))
    else:
        df_tracks = pd.DataFrame()
    
    # Fetch and organize Top Artists
    print("\n\n🎤 FETCHING TOP ARTISTS (WITH COUNTRIES)")
    print("-" * 60)
    top_artists = get_top_artists(limit=50)
    if top_artists:
        df_artists = organize_top_artists(top_artists)
        print(f"   organized {len(df_artists)} artists")
        print(f"\n   Sample:")
        print(df_artists[['rank', 'artist_name', 'country', 'listeners']].head())
    else:
        df_artists = pd.DataFrame()
    
    # Fetch and organize Top Tags/Genres
    print("\n\n🎼 FETCHING TOP GENRES")
    print("-" * 60)
    top_tags = get_top_tags(limit=100)
    if top_tags:
        df_tags = organize_top_tags(top_tags)
        print(f"   organized {len(df_tags)} genres")
        print(f"\n   Sample:")
        print(df_tags[['rank', 'genre', 'reach']].head())
    else:
        df_tags = pd.DataFrame()
    
    print("\n" + "=" * 60)
    print("✅ Pipeline completed!")
    print("=" * 60)
    
    return df_tracks, df_artists, df_tags

if __name__ == '__main__':
    df_tracks, df_artists, df_genres = main()

Last.fm Global Data → DataFrames

🎵 FETCHING TOP TRACKS
------------------------------------------------------------
   Fetching top 100 global tracks...
      Fetching genres for The Fate of Ophelia... ✓
      Fetching genres for Last Christmas... ✓
      Fetching genres for back to friends... ✓
      Fetching genres for End of Beginning... ✓
      Fetching genres for No One Noticed... ✓
      Fetching genres for Sienna... ✓
      Fetching genres for Good Luck, Babe!... ✓
      Fetching genres for Manchild... ✓
      Fetching genres for Creep... ✓
      Fetching genres for Tears... ✓
      Fetching genres for WHERE IS MY HUSBAND!... ✓
      Fetching genres for The Less I Know the Better... ✓
      Fetching genres for All I Want for Christmas Is You... ✓
      Fetching genres for Let Down... ✓
      Fetching genres for Dracula... ✓
      Fetching genres for BIRDS OF A FEATHER... ✓
      Fetching genres for Fame Is a Gun... ✓
      Fetching genres for Gabriela... ✓
      Fetching genres

In [192]:
df_tracks

,rank,track_name,artist,genres,playcount,listeners,url,data_type,extracted_at
0,1,The Fate of Ophelia,Taylor Swift,"pop, best of 2025, dance-pop, power pop, synthpop",15914357,779603,https://www.last.fm/music/Taylor+Swift/_/The+F...,top_track,2025-12-10 15:37:43.689593
1,2,Last Christmas,Wham!,"christmas, 80s, pop, xmas, Wham",14371272,1912402,https://www.last.fm/music/Wham%21/_/Last+Chris...,top_track,2025-12-10 15:37:44.072923
2,3,back to friends,sombr,"WSUM 91.7 FM Madison, rock and roll over, The ...",18617293,1178817,https://www.last.fm/music/sombr/_/back+to+friends,top_track,2025-12-10 15:37:44.546190
3,4,End of Beginning,Djo,"indie, pop, best of 2022, indie pop, synth-pop",27013875,1666351,https://www.last.fm/music/Djo/_/End+of+Beginning,top_track,2025-12-10 15:37:45.272884
4,5,No One Noticed,The Marías,"dream pop, Neo-Psychedelia, bedroom pop, Sophi...",27765505,1381982,https://www.last.fm/music/The+Mar%C3%ADas/_/No...,top_track,2025-12-10 15:37:45.641749
...,...,...,...,...,...,...,...,...,...
95,96,Bed Chem,Sabrina Carpenter,"pop, 2024, joaoaksnes, funk, female vocalist",23701781,1174845,https://www.last.fm/music/Sabrina+Carpenter/_/...,top_track,2025-12-10 15:38:50.263905
96,97,Sailor Song,Gigi Perez,"WSUM 91.7 FM Madison, my top songs, rock and r...",24128007,1128007,https://www.last.fm/music/Gigi+Perez/_/Sailor+...,top_track,2025-12-10 15:38:50.642027
97,98,Sextape,Deftones,"alternative metal, metal, alternative rock, De...",25344207,1428101,https://www.last.fm/music/Deftones/_/Sextape,top_track,2025-12-10 15:38:51.005347
98,99,My Own Summer (Shove It),Deftones,"Nu Metal, metal, alternative, alternative meta...",24775118,1721071,https://www.last.fm/music/Deftones/_/My+Own+Su...,top_track,2025-12-10 15:38:51.365760


In [193]:
df_tracks['genres'] = df_tracks['genres'].str.split(', ')
df_tracks = df_tracks.explode('genres')

In [194]:
df_tracks

,rank,track_name,artist,genres,playcount,listeners,url,data_type,extracted_at
0,1,The Fate of Ophelia,Taylor Swift,pop,15914357,779603,https://www.last.fm/music/Taylor+Swift/_/The+F...,top_track,2025-12-10 15:37:43.689593
0,1,The Fate of Ophelia,Taylor Swift,best of 2025,15914357,779603,https://www.last.fm/music/Taylor+Swift/_/The+F...,top_track,2025-12-10 15:37:43.689593
0,1,The Fate of Ophelia,Taylor Swift,dance-pop,15914357,779603,https://www.last.fm/music/Taylor+Swift/_/The+F...,top_track,2025-12-10 15:37:43.689593
0,1,The Fate of Ophelia,Taylor Swift,power pop,15914357,779603,https://www.last.fm/music/Taylor+Swift/_/The+F...,top_track,2025-12-10 15:37:43.689593
0,1,The Fate of Ophelia,Taylor Swift,synthpop,15914357,779603,https://www.last.fm/music/Taylor+Swift/_/The+F...,top_track,2025-12-10 15:37:43.689593
...,...,...,...,...,...,...,...,...,...
99,100,Guess featuring billie eilish,Charli xcx,electro house,27274509,1478723,https://www.last.fm/music/Charli+xcx/_/Guess+f...,top_track,2025-12-10 15:38:51.745255
99,100,Guess featuring billie eilish,Charli xcx,tech house,27274509,1478723,https://www.last.fm/music/Charli+xcx/_/Guess+f...,top_track,2025-12-10 15:38:51.745255
99,100,Guess featuring billie eilish,Charli xcx,peter,27274509,1478723,https://www.last.fm/music/Charli+xcx/_/Guess+f...,top_track,2025-12-10 15:38:51.745255
99,100,Guess featuring billie eilish,Charli xcx,House,27274509,1478723,https://www.last.fm/music/Charli+xcx/_/Guess+f...,top_track,2025-12-10 15:38:51.745255


In [195]:
df_artists

,rank,artist_name,country,playcount,listeners,url,data_type,extracted_at
0,1,Ariana Grande,None,1023968675,4263747,https://www.last.fm/music/Ariana+Grande,top_artist,2025-12-10 15:38:52.435897
1,2,The Weeknd,None,1048650698,5100753,https://www.last.fm/music/The+Weeknd,top_artist,2025-12-10 15:38:52.807800
2,3,Radiohead,None,1296799476,7998869,https://www.last.fm/music/Radiohead,top_artist,2025-12-10 15:38:53.232102
3,4,Kendrick Lamar,None,974667129,4898100,https://www.last.fm/music/Kendrick+Lamar,top_artist,2025-12-10 15:38:53.677101
4,5,Taylor Swift,None,3486190045,5732609,https://www.last.fm/music/Taylor+Swift,top_artist,2025-12-10 15:38:54.053056
5,6,"Tyler, The Creator",None,987619941,4139819,"https://www.last.fm/music/Tyler,+The+Creator",top_artist,2025-12-10 15:38:54.465884
6,7,Sabrina Carpenter,None,570614982,3037165,https://www.last.fm/music/Sabrina+Carpenter,top_artist,2025-12-10 15:38:55.160385
7,8,Lady Gaga,None,976878639,7486737,https://www.last.fm/music/Lady+Gaga,top_artist,2025-12-10 15:38:55.528516
8,9,Kanye West,None,1442212258,7740246,https://www.last.fm/music/Kanye+West,top_artist,2025-12-10 15:38:55.890076
9,10,Drake,None,1072445085,6523111,https://www.last.fm/music/Drake,top_artist,2025-12-10 15:38:56.328020


In [196]:
df_genres


,rank,genre,reach,url,data_type,extracted_at
0,1,rock,402594,https://www.last.fm/tag/rock,top_genre,2025-12-10 15:39:12.333293
1,2,electronic,261868,https://www.last.fm/tag/electronic,top_genre,2025-12-10 15:39:12.333308
2,3,seen live,82553,https://www.last.fm/tag/seen+live,top_genre,2025-12-10 15:39:12.333314
3,4,alternative,267006,https://www.last.fm/tag/alternative,top_genre,2025-12-10 15:39:12.333319
4,5,pop,233508,https://www.last.fm/tag/pop,top_genre,2025-12-10 15:39:12.333323
...,...,...,...,...,...,...
95,96,Nu Metal,50436,https://www.last.fm/tag/Nu+Metal,top_genre,2025-12-10 15:39:12.333606
96,97,j-pop,26773,https://www.last.fm/tag/j-pop,top_genre,2025-12-10 15:39:12.333608
97,98,albums I own,14038,https://www.last.fm/tag/albums+I+own,top_genre,2025-12-10 15:39:12.333609
98,99,under 2000 listeners,6403,https://www.last.fm/tag/under+2000+listeners,top_genre,2025-12-10 15:39:12.333611


## **Data Cleaning**

In [197]:
df_tracks[['track_name', 'artist', 'genres']].head()
df_tracks['genres'][0]

0             pop
0    best of 2025
0       dance-pop
0       power pop
0        synthpop
Name: genres, dtype: object

In [198]:
df_tracks['genres'] = df_tracks['genres'].astype(str)

cleaned_genres = []

for g in df_tracks['genres']:
    text = str(g).strip().lower()
    if text == "" or text == "none" or text == "nan":
        cleaned_genres.append(None)
    else:
        cleaned_genres.append(text)

df_tracks['genres_clean'] = cleaned_genres


In [199]:
df_tracks = df_tracks.drop_duplicates(
    subset=['track_name', 'artist', 'genres_clean']
)


In [200]:
import pandas as pd

track_genres_dict = {}

for i in range(len(df_tracks)):
    track = df_tracks.iloc[i]['track_name']
    artist = df_tracks.iloc[i]['artist']
    genre = df_tracks.iloc[i]['genres_clean']

    key = (track, artist)

    if key not in track_genres_dict:
        track_genres_dict[key] = []

    if genre is not None and genre not in track_genres_dict[key]:
        track_genres_dict[key].append(genre)


In [201]:
rows = []

for key in track_genres_dict:
    track, artist = key
    genres_list = track_genres_dict[key]

    if len(genres_list) > 0:
        genres_str = ", ".join(genres_list)
        primary_genre = genres_list[0]
    else:
        genres_str = None
        primary_genre = "unknown"

    rows.append({
        'track_name': track,
        'artist': artist,
        'genres_list': genres_list,
        'genres_str': genres_str,
        'primary_genre': primary_genre
    })

df_tracks_clean = pd.DataFrame(rows)


In [202]:
df_tracks_clean.head()
df_tracks_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   track_name     100 non-null    object
 1   artist         100 non-null    object
 2   genres_list    100 non-null    object
 3   genres_str     100 non-null    object
 4   primary_genre  100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB


In [203]:
df_artists.head()
df_artists.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   rank          50 non-null     int64         
 1   artist_name   50 non-null     object        
 2   country       0 non-null      object        
 3   playcount     50 non-null     int64         
 4   listeners     50 non-null     int64         
 5   url           50 non-null     object        
 6   data_type     50 non-null     object        
 7   extracted_at  50 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 3.3+ KB


In [204]:
df_artists_clean = df_artists.copy()


In [205]:
df_artists_clean['rank'] = pd.to_numeric(df_artists_clean['rank'], errors='coerce')
df_artists_clean['listeners'] = pd.to_numeric(df_artists_clean['listeners'], errors='coerce')


In [206]:
df_artists_clean = df_artists_clean.dropna(subset=['rank'])


In [207]:
df_artists_clean['listeners'] = df_artists_clean['listeners'].fillna(0)


In [208]:
df_artists_clean['artist_name'] = df_artists_clean['artist_name'].astype(str).str.strip()


In [209]:
cleaned_country = []

for c in df_artists_clean['country']:
    if c is None:
        cleaned_country.append("Unknown")
    else:
        text = str(c).strip()
        if text == "":
            cleaned_country.append("Unknown")
        else:
            cleaned_country.append(text)

df_artists_clean['country'] = cleaned_country


In [210]:
df_artists_clean.head()
df_artists_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   rank          50 non-null     int64         
 1   artist_name   50 non-null     object        
 2   country       50 non-null     object        
 3   playcount     50 non-null     int64         
 4   listeners     50 non-null     int64         
 5   url           50 non-null     object        
 6   data_type     50 non-null     object        
 7   extracted_at  50 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 3.3+ KB


In [211]:
df_genres.head()
df_genres.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   rank          100 non-null    int64         
 1   genre         100 non-null    object        
 2   reach         100 non-null    int64         
 3   url           100 non-null    object        
 4   data_type     100 non-null    object        
 5   extracted_at  100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 4.8+ KB


In [212]:
df_genres_clean = df_genres.copy()


In [213]:
df_genres_clean['rank'] = pd.to_numeric(df_genres_clean['rank'], errors='coerce')
df_genres_clean['reach'] = pd.to_numeric(df_genres_clean['reach'], errors='coerce')


In [214]:
df_genres_clean = df_genres_clean.dropna(subset=['rank'])


In [215]:
df_genres_clean['reach'] = df_genres_clean['reach'].fillna(0)


In [216]:
cleaned_genre = []

for g in df_genres_clean['genre']:
    if g is None:
        cleaned_genre.append(None)
    else:
        cleaned_genre.append(str(g).strip().lower())

df_genres_clean['genre'] = cleaned_genre


In [217]:
df_genres_clean = df_genres_clean.drop_duplicates(subset=['genre'])


In [218]:
df_genres_clean.head()
df_genres_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   rank          100 non-null    int64         
 1   genre         100 non-null    object        
 2   reach         100 non-null    int64         
 3   url           100 non-null    object        
 4   data_type     100 non-null    object        
 5   extracted_at  100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 4.8+ KB


In [219]:
df_tracks_bq = df_tracks_clean.drop(columns=['genres_list'])

In [220]:
df_tracks_bq.head()
df_tracks_bq.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   track_name     100 non-null    object
 1   artist         100 non-null    object
 2   genres_str     100 non-null    object
 3   primary_genre  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


## **Upload Big Query**

In [221]:
df_tracks_bq = df_tracks.merge(
    df_artists[['artist_name', 'playcount', 'listeners']],
    left_on='artist',
    right_on='artist_name',
    how='left'
)

df_tracks_bq.drop(columns=['artist_name'], inplace=True)

print(df_tracks_bq.info())
print(df_tracks_bq.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   rank          481 non-null    int64         
 1   track_name    481 non-null    object        
 2   artist        481 non-null    object        
 3   genres        481 non-null    object        
 4   playcount_x   481 non-null    int64         
 5   listeners_x   481 non-null    int64         
 6   url           481 non-null    object        
 7   data_type     481 non-null    object        
 8   extracted_at  481 non-null    datetime64[ns]
 9   genres_clean  481 non-null    object        
 10  playcount_y   292 non-null    float64       
 11  listeners_y   292 non-null    float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 45.2+ KB
None
   rank           track_name        artist        genres  playcount_x  \
0     1  The Fate of Ophelia  Taylo

In [222]:
df_artists_bq = df_artists_clean.copy()
df_artists_bq

,rank,artist_name,country,playcount,listeners,url,data_type,extracted_at
0,1,Ariana Grande,Unknown,1023968675,4263747,https://www.last.fm/music/Ariana+Grande,top_artist,2025-12-10 15:38:52.435897
1,2,The Weeknd,Unknown,1048650698,5100753,https://www.last.fm/music/The+Weeknd,top_artist,2025-12-10 15:38:52.807800
2,3,Radiohead,Unknown,1296799476,7998869,https://www.last.fm/music/Radiohead,top_artist,2025-12-10 15:38:53.232102
3,4,Kendrick Lamar,Unknown,974667129,4898100,https://www.last.fm/music/Kendrick+Lamar,top_artist,2025-12-10 15:38:53.677101
4,5,Taylor Swift,Unknown,3486190045,5732609,https://www.last.fm/music/Taylor+Swift,top_artist,2025-12-10 15:38:54.053056
5,6,"Tyler, The Creator",Unknown,987619941,4139819,"https://www.last.fm/music/Tyler,+The+Creator",top_artist,2025-12-10 15:38:54.465884
6,7,Sabrina Carpenter,Unknown,570614982,3037165,https://www.last.fm/music/Sabrina+Carpenter,top_artist,2025-12-10 15:38:55.160385
7,8,Lady Gaga,Unknown,976878639,7486737,https://www.last.fm/music/Lady+Gaga,top_artist,2025-12-10 15:38:55.528516
8,9,Kanye West,Unknown,1442212258,7740246,https://www.last.fm/music/Kanye+West,top_artist,2025-12-10 15:38:55.890076
9,10,Drake,Unknown,1072445085,6523111,https://www.last.fm/music/Drake,top_artist,2025-12-10 15:38:56.328020


In [223]:
df_genres_bq = df_genres_clean.copy()
df_genres_bq

,rank,genre,reach,url,data_type,extracted_at
0,1,rock,402594,https://www.last.fm/tag/rock,top_genre,2025-12-10 15:39:12.333293
1,2,electronic,261868,https://www.last.fm/tag/electronic,top_genre,2025-12-10 15:39:12.333308
2,3,seen live,82553,https://www.last.fm/tag/seen+live,top_genre,2025-12-10 15:39:12.333314
3,4,alternative,267006,https://www.last.fm/tag/alternative,top_genre,2025-12-10 15:39:12.333319
4,5,pop,233508,https://www.last.fm/tag/pop,top_genre,2025-12-10 15:39:12.333323
...,...,...,...,...,...,...
95,96,nu metal,50436,https://www.last.fm/tag/Nu+Metal,top_genre,2025-12-10 15:39:12.333606
96,97,j-pop,26773,https://www.last.fm/tag/j-pop,top_genre,2025-12-10 15:39:12.333608
97,98,albums i own,14038,https://www.last.fm/tag/albums+I+own,top_genre,2025-12-10 15:39:12.333609
98,99,under 2000 listeners,6403,https://www.last.fm/tag/under+2000+listeners,top_genre,2025-12-10 15:39:12.333611


In [224]:
key_path = "d:\\Purwadhika\\Portofolio\\portofolio-2-480507-784a4cbb0d43.json"

In [225]:
project_id = "portofolio-2-480507"

In [226]:
dataset_id = "data_lastfm_porto2"

In [227]:
pip install pandas_gbq

Note: you may need to restart the kernel to use updated packages.


In [228]:
from google.oauth2 import service_account
import pandas_gbq

In [229]:
# Konfigurasi Bigquery

# Setup Credentials
scopes = ["https://www.googleapis.com/auth/bigquery"]
credentials = service_account.Credentials.from_service_account_file(key_path,
                                                                    scopes=scopes)

In [230]:
print("columns before:", df_tracks_bq.columns.tolist())

if 'playcount_x' in df_tracks_bq.columns:
    pc_x = df_tracks_bq['playcount_x']
else:
    pc_x = pd.Series(np.nan, index=df_tracks_bq.index)

pc_y = df_tracks_bq['playcount_y'] if 'playcount_y' in df_tracks_bq.columns else pd.Series(np.nan, index=df_tracks_bq.index)

if 'listeners_x' in df_tracks_bq.columns:
    li_x = df_tracks_bq['listeners_x']
else:
    li_x = pd.Series(np.nan, index=df_tracks_bq.index)

li_y = df_tracks_bq['listeners_y'] if 'listeners_y' in df_tracks_bq.columns else pd.Series(np.nan, index=df_tracks_bq.index)

df_tracks_bq['playcount'] = pc_x.fillna(pc_y).fillna(0)
df_tracks_bq['listeners'] = li_x.fillna(li_y).fillna(0)

df_tracks_bq['playcount'] = pd.to_numeric(df_tracks_bq['playcount'], errors='coerce').fillna(0).astype('int64')
df_tracks_bq['listeners'] = pd.to_numeric(df_tracks_bq['listeners'], errors='coerce').fillna(0).astype('int64')

for c in ['playcount_x','playcount_y','listeners_x','listeners_y']:
    if c in df_tracks_bq.columns:
        df_tracks_bq.drop(columns=[c], inplace=True)

print("\ncolumns after:", df_tracks_bq.columns.tolist())
print("\ndtypes:\n", df_tracks_bq.dtypes)
print("\npreview (track_name, artist, playcount, listeners):")
print(df_tracks_bq[['track_name','artist','playcount','listeners']].head(10).to_string(index=False))
print("\nrows:", len(df_tracks_bq))

df_tracks_bq['popularity_score'] = (df_tracks_bq['playcount'] * 0.6) + (df_tracks_bq['listeners'] * 0.4)

print("\npreview with popularity_score:")
print(df_tracks_bq[['track_name','artist','playcount','listeners','popularity_score']].head(8).to_string(index=False))


columns before: ['rank', 'track_name', 'artist', 'genres', 'playcount_x', 'listeners_x', 'url', 'data_type', 'extracted_at', 'genres_clean', 'playcount_y', 'listeners_y']

columns after: ['rank', 'track_name', 'artist', 'genres', 'url', 'data_type', 'extracted_at', 'genres_clean', 'playcount', 'listeners']

dtypes:
 rank                     int64
track_name              object
artist                  object
genres                  object
url                     object
data_type               object
extracted_at    datetime64[ns]
genres_clean            object
playcount                int64
listeners                int64
dtype: object

preview (track_name, artist, playcount, listeners):
         track_name       artist  playcount  listeners
The Fate of Ophelia Taylor Swift   15914357     779603
The Fate of Ophelia Taylor Swift   15914357     779603
The Fate of Ophelia Taylor Swift   15914357     779603
The Fate of Ophelia Taylor Swift   15914357     779603
The Fate of Ophelia Taylor Swi

In [ ]:
pandas_gbq.to_gbq(
    dataframe=df_artists_bq,
    destination_table=f"{dataset_id}.artists",
    project_id=project_id,
    if_exists="replace",
    credentials=credentials
)

100%|██████████| 1/1 [00:00<00:00, 14614.30it/s]


In [ ]:
pandas_gbq.to_gbq(
    dataframe=df_genres_bq,
    destination_table=f"{dataset_id}.genres",
    project_id=project_id,
    if_exists="replace",
    credentials=credentials
)

100%|██████████| 1/1 [00:00<00:00, 18001.30it/s]


In [233]:
df_tracks_bq.to_gbq(
    destination_table='data_lastfm_porto2.tracks',
    project_id='portofolio-2-480507',
    if_exists='replace'
)


100%|██████████| 1/1 [00:00<00:00, 17403.75it/s]
